# Retrieval Augmented Generation

## Import packages

In [1]:
import os

In [2]:
import langchain
import rootutils
from dotenv import find_dotenv, load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import PyPDFLoader, WikipediaLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant

## Settings

In [3]:
# read local .env file
_ = load_dotenv(find_dotenv())

In [4]:
class debug_langchain:
    def __init__(self, enable: bool = True):
        self.enable = enable

    def __enter__(self):
        langchain.debug = True if self.enable else False

    def __exit__(self, exc_type, exc_val, exc_tb):
        langchain.debug = False

In [5]:
SEED = 42
SENTENCE_ENDINGS = [".", "!", "?"]
WORDS_BREAKS = list(reversed([",", ";", ":", " ", "(", ")", "[", "]", "{", "}", "\t", "\n"]))

In [6]:
path_to_root = rootutils.find_root(indicator=".project-root")
path_to_data = path_to_root / "data"

path_to_data.mkdir(exist_ok=True, parents=True)

In [7]:
llm_deployment_name = os.getenv("LLM_DEPLOYMENT_NAME")
embedding_deployment_name = os.getenv("EMBEDDING_DEPLOYMENT_NAME")

## Prepare data

In [8]:
# Load PDF
loaders = (
    [
        # Duplicate documents on purpose - messy data
        PyPDFLoader(file_path=str(path_to_data / file_name))
        for file_name in os.listdir(path_to_data)
        if file_name.endswith(".pdf")
    ]
    + [
        WikipediaLoader(query="Розпізнавання іменованих сутностей", load_max_docs=2, lang="uk"),
        WikipediaLoader(query="Нейронні мережі", load_max_docs=2, lang="uk"),
        WikipediaLoader(query="Дід Панас", load_max_docs=1, lang="uk"),
    ]
    + [
        WikipediaLoader(query="Messi", load_max_docs=2, lang="en"),
        WikipediaLoader(query="Дід Панас", load_max_docs=1, lang="en"),
    ]
)

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [9]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=768,  # 2048
    chunk_overlap=128,  # 128
    separators=SENTENCE_ENDINGS + WORDS_BREAKS,
)

In [10]:
splits = text_splitter.split_documents(docs)

In [11]:
len(splits)

170

## Create embeddings and fill vector store

In [12]:
embeddings = OpenAIEmbeddings(deployment=embedding_deployment_name)

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
url = os.getenv("QDRANT_URL")
# collection_name = os.getenv("QDRANT_COLLECTION_NAME")
collection_name = "my_custom_documents"

qdrant = Qdrant.from_documents(
    splits,
    embeddings,
    url=url,
    collection_name=collection_name,
)

## Try out the search

In [14]:
query = "What is Bias-Variance Tradeoff?"
found_docs = qdrant.similarity_search(query)
found_docs[0]

Document(page_content='CS229 Bias-Variance and Error Analysis\nYoann Le Calonnec\nOctober 2, 2017\n1 The Bias-Variance Tradeoﬀ\nAssume you are given a well ﬁtted machine learning model ˆfthat you want to apply on\nsome test dataset. For instance, the model could be a linear regression whose parameters\nwere computed using some training set diﬀerent from your test set. For each point xin your\ntest set, you want to predict the associated target y∈R, and compute the mean squared\nerror (MSE)\nE(x,y)∼test set|ˆf(x)−y|2\nYou now realize that this MSE is too high, and try to ﬁnd an explanation to this result:\n•Overﬁtting: the model is too closely related to the examples in the training set and\ndoesn’t generalize well to other examples.\n•Underﬁtting: the model didn’t gather enough information from the training set, and\ndoesn’t capture the link between the features xand the target y.\n•The data is simply noisy, that is the model is neither overﬁtting or underﬁtting, and\nthe high MSE is s

In [15]:
query = "Що таке розпізнавання іменованих сутностей?"
found_docs = qdrant.similarity_search(query)
found_docs[0]

Document(page_content="У обробці природної мови зв'язування сутностей (англ. Entity Linking), яке також називають зв'язування іменованих сутностей (ЗІС), розпізнавання іменованих сутностей (РІС), розпізнавання і неоднозначності іменованих сутностей (РНІС) або нормалізація іменованих сутностей (НІС), — це присвоєння унікальної ідентичності об'єктам (наприклад, відомим особам, місцям чи компаніям), що згадуються у тексті. Наприклад, розглянемо речення «Дніпро — річка України». Ідея полягає в тому, щоб визначити, що «Дніпро» належить до річки Дніпро, а не до Дніпро-арена чи будь-якої іншої організації, яку можна назвати «Дніпро». Зв'язування сутностей відрізняється від розпізнавання іменованих сутностей (РІС) тим, що РІС ідентифікує появу іменованої сутності в тексті, але не визначає, що саме це за сутність (див. Відмінності від інших технік).\n\n\n== Вступ ==\nУ зв'язуванні сутностей слова, що цікавлять (імена осіб, місцеположення та компанії) відображаються з вхідного тексту до відповід

In [16]:
query = "Хто такий дід Панас?"
found_docs = qdrant.max_marginal_relevance_search(query, k=2, fetch_k=10)
found_docs[0]

Document(page_content=".\n1941 — призваний у військовий театр Південно-Західного фронту, потрапив у полон, але втік. Згодом потрапив до окупованого гітлерівцями Києва, влаштувався на роботу на залізниці, де організував драматичний гурток. 1946 року був скерований до Луцька — у Волинський український музично-драматичний театр імені Тараса Шевченка, де працював до 1959-го.\nУ Луцьку одружився із секретаркою театру Галиною, де народився його син Богдан.1959—1982 — на Київській кіностудії художніх фільмів імені Олександра Довженка.\nЗ 1962 р. у ролі діда Панаса розповідав на Українському радіо «вечірню казку» для дітей (перед тим Вескляров працював у цій ролі на підміні іншого актора, якого згодом замінив цілком).1964—1986 — у ролі діда Панаса вів на Українському телебаченні програму «На добраніч, діти».\n\n\n== Вшанування пам'яті ==\nПохований у колумбарії Байкового кладовища. Вдова виїхала до США, перед тим передавши кіноплівки із записами «діда Панаса» братам Капрановим.\n\n2019 року на

## Create simple RAG chain using LlamaCpp

In [17]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from qdrant_client import QdrantClient

In [18]:
doc_store = Qdrant(
    client=QdrantClient(url=url),
    collection_name=collection_name,
    embeddings=embeddings,
)

In [19]:
llm = AzureChatOpenAI(
    deployment_name=llm_deployment_name,
    temperature=0,
)

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=doc_store.as_retriever(),
    # retriever=qdrant.as_retriever(search_type="mmr"),
    return_source_documents=False,
    # chain_type_kwargs={"prompt": custom_prompt_template},
)

In [21]:
with debug_langchain():
    question = "What is Gaussian kernel?"
    # qa_chain({"query": question})
    qa_chain.run(question)

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is Gaussian kernel?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is Gaussian kernel?",
  "context": "9\na feature map φsuch that the kernel Kdeﬁned above satisﬁes K(x,z) =\nφ(x)Tφ(z)? Inthisparticularexample, theanswerisyes. Thiskernel iscalled\ntheGaussian kernel , and corresponds to an inﬁnite dimensional feature\nmapping φ. We will give a precise characterization about what propert ies\na function Kneeds to satisfy so that it can be a valid kernel function that\ncorresponds to some feature map φ.\nNecessary conditions for valid kernels. Suppose for now that Kis\nindeed a valid kernel corresponding to some feature mapping φ, and we will\nﬁrst see what properties it satisﬁes. Now, consider some ﬁnit e 

In [22]:
with debug_langchain():
    question = "Who is Grandpa Panas?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Who is Grandpa Panas?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Who is Grandpa Panas?",
  "context": "Petro Yukhymovych Vesklyarov (Ukrainian: Вескляров Петро Юхимович) (June 10 [O.S. May 28] 1911 in Talne, Ukraine – January 5, 1994 in Kyiv) was a Ukrainian theater and television actor. He was also known by the nickname Did Panas (Grandpa Panas, Ukrainian: дід Панас).\nBetween 1932 and 1940, Vesklyarov was an actor in a travelling workers' theatre, and between 1946 and 1959 he performed at the Taras Shevchenko Musical-Drama Theatre in Lutsk, Volyn. Between 1959 and 1982 Veslklyarov worked in the Dovzhenko Film Studios, appearing in a number of films. He starred in the 1959 drama film Ivanna and appeared in the

In [23]:
with debug_langchain():
    question = "Хто такий дід Панас?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Хто такий дід Панас?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Хто такий дід Панас?",
  "context": ".\n1941 — призваний у військовий театр Південно-Західного фронту, потрапив у полон, але втік. Згодом потрапив до окупованого гітлерівцями Києва, влаштувався на роботу на залізниці, де організував драматичний гурток. 1946 року був скерований до Луцька — у Волинський український музично-драматичний театр імені Тараса Шевченка, де працював до 1959-го.\nУ Луцьку одружився із секретаркою театру Галиною, де народився його син Богдан.1959—1982 — на Київській кіностудії художніх фільмів імені Олександра Довженка.\nЗ 1962 р. у ролі діда Панаса розповідав на Українському радіо «вечірню казку» для дітей (перед тим Вескляро

## Modify the chain to use custom prompt in English

In [24]:
custom_prompt = """Use the following pieces of context to answer the question at the end. Please provide a short single-sentence summary answer only. If you don't know the answer or if it's not present in given context, don't try to make up an answer.
Context: {context}
Question: {question}
Helpful Answer:
"""
custom_prompt_template = PromptTemplate(template=custom_prompt, input_variables=["context", "question"])

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=doc_store.as_retriever(),
    # retriever=qdrant.as_retriever(search_type="mmr"),
    return_source_documents=False,
    chain_type_kwargs={"prompt": custom_prompt_template},
)

In [26]:
with debug_langchain():
    question = "What is Gaussian kernel?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is Gaussian kernel?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is Gaussian kernel?",
  "context": "9\na feature map φsuch that the kernel Kdeﬁned above satisﬁes K(x,z) =\nφ(x)Tφ(z)? Inthisparticularexample, theanswerisyes. Thiskernel iscalled\ntheGaussian kernel , and corresponds to an inﬁnite dimensional feature\nmapping φ. We will give a precise characterization about what propert ies\na function Kneeds to satisfy so that it can be a valid kernel function that\ncorresponds to some feature map φ.\nNecessary conditions for valid kernels. Suppose for now that Kis\nindeed a valid kernel corresponding to some feature mapping φ, and we will\nﬁrst see what properties it satisﬁes. Now, consider some ﬁnit e 

In [27]:
with debug_langchain():
    question = "Who is Grandpa Panas?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Who is Grandpa Panas?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Who is Grandpa Panas?",
  "context": "Petro Yukhymovych Vesklyarov (Ukrainian: Вескляров Петро Юхимович) (June 10 [O.S. May 28] 1911 in Talne, Ukraine – January 5, 1994 in Kyiv) was a Ukrainian theater and television actor. He was also known by the nickname Did Panas (Grandpa Panas, Ukrainian: дід Панас).\nBetween 1932 and 1940, Vesklyarov was an actor in a travelling workers' theatre, and between 1946 and 1959 he performed at the Taras Shevchenko Musical-Drama Theatre in Lutsk, Volyn. Between 1959 and 1982 Veslklyarov worked in the Dovzhenko Film Studios, appearing in a number of films. He starred in the 1959 drama film Ivanna and appeared in the

In [28]:
with debug_langchain():
    question = "Хто такий дід Панас?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Хто такий дід Панас?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Хто такий дід Панас?",
  "context": ".\n1941 — призваний у військовий театр Південно-Західного фронту, потрапив у полон, але втік. Згодом потрапив до окупованого гітлерівцями Києва, влаштувався на роботу на залізниці, де організував драматичний гурток. 1946 року був скерований до Луцька — у Волинський український музично-драматичний театр імені Тараса Шевченка, де працював до 1959-го.\nУ Луцьку одружився із секретаркою театру Галиною, де народився його син Богдан.1959—1982 — на Київській кіностудії художніх фільмів імені Олександра Довженка.\nЗ 1962 р. у ролі діда Панаса розповідав на Українському радіо «вечірню казку» для дітей (перед тим Вескляро

In [29]:
with debug_langchain():
    question = "Що таке розпізнавання іменованих сутностей?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Що таке розпізнавання іменованих сутностей?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Що таке розпізнавання іменованих сутностей?",
  "context": "У обробці природної мови зв'язування сутностей (англ. Entity Linking), яке також називають зв'язування іменованих сутностей (ЗІС), розпізнавання іменованих сутностей (РІС), розпізнавання і неоднозначності іменованих сутностей (РНІС) або нормалізація іменованих сутностей (НІС), — це присвоєння унікальної ідентичності об'єктам (наприклад, відомим особам, місцям чи компаніям), що згадуються у тексті. Наприклад, розглянемо речення «Дніпро — річка України». Ідея полягає в тому, щоб визначити, що «Дніпро» належить до річки Дніпро, а не до Дніпро-арена чи будь-якої іншої ор

## Modify the chain to use custom prompt in English and context compression

In [30]:
# llama
custom_retriever_prompt = """Given the following question and context, extract any part of the context AS IS that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT. Remember, DO NOT edit the extracted parts of the context.
Question: {question}
Context: {context}
Answer:
"""
custom_retriever_prompt_template = PromptTemplate(
    template=custom_retriever_prompt, input_variables=["question", "context"]
)

In [31]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [32]:
k = 5

base_compressor = LLMChainExtractor.from_llm(llm=llm, prompt=custom_retriever_prompt_template)
# base_retriever = doc_store.as_retriever()
base_retriever = doc_store.as_retriever(search_kwargs={"k": k})

compression_retriever = ContextualCompressionRetriever(
    base_compressor=base_compressor,
    base_retriever=base_retriever,
    k=k,
)

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff", "map_reduce", "map_rerank", and "refine".
    retriever=compression_retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": custom_prompt_template},
)

In [34]:
with debug_langchain():
    question = "What is Gaussian kernel?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is Gaussian kernel?"
}
[chain/start] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is Gaussian kernel?",
  "context": "9\na feature map φsuch that the kernel Kdeﬁned above satisﬁes K(x,z) =\nφ(x)Tφ(z)? Inthisparticularexample, theanswerisyes. Thiskernel iscalled\ntheGaussian kernel , and corresponds to an inﬁnite dimensional feature\nmapping φ. We will give a precise characterization about what propert ies\na function Kneeds to satisfy so that it can be a valid kernel function that\ncorresponds to some feature map φ.\nNecessary conditions for valid kernels. Suppose for now that Kis\nindeed a valid kernel corresponding to some feature mapping φ, and we will\nﬁrst see what properties it satisﬁes. Now, consider some ﬁnit e set ofnpoints\n(not necessarily the training set) {x(1),...,x(n)}, and let a square, n-by-n\nmatrixKbe deﬁned so

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain > 5:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Gaussian kernel is a valid kernel that corresponds to an infinite dimensional feature mapping φ.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The Gaussian kernel is a valid kernel that corresponds to an infinite dimensional feature mapping φ.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 638,
      "completion_tokens": 17,
      "total_tokens": 655
    },
    "model_name": "gpt-35-turbo",
    "system_fing

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 6:chain:LLMChain > 7:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Kernels can be directly applied to support vector machines. Kernels have a significantly broader applicability than linear regression and SVMs. Kernels can be applied with the perceptron to derive a kernel perceptron algorithm. Kernels give a way to apply SVMs efficiently in very high dimensional feature spaces.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Kernels can be directly applied to support vector machines. Kernels have a significantly broader applicability than linear regression and SVMs. Kern

...

[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 8:chain:LLMChain > 9:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "K(x,z) =⟨φ(x),φ(z)⟩is the kernel function that corresponds to the the feature mapping φgiven (shown here for the case of d= 3).",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "K(x,z) =⟨φ(x),φ(z)⟩is the kernel function that corresponds to the the feature mapping φgiven (shown here for the case of d= 3).",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 475,
      "completion_tokens": 39,
      "total_tokens": 514
   

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 10:chain:LLMChain > 11:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Gaussian kernel was used in SVMs for the digit recognition problem and was able to obtain extremely good performance.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The Gaussian kernel was used in SVMs for the digit recognition problem and was able to obtain extremely good performance.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 722,
      "completion_tokens": 22,
      "total_tokens": 744
    },
    "m

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 12:chain:LLMChain > 13:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Gaussian kernel is not explicitly mentioned in the context, so the output is NO_OUTPUT.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The Gaussian kernel is not explicitly mentioned in the context, so the output is NO_OUTPUT.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 686,
      "completion_tokens": 18,
      "total_tokens": 704
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprint": ""
  }

In [35]:
with debug_langchain():
    question = "Who is Grandpa Panas?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Who is Grandpa Panas?"
}
[chain/start] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Who is Grandpa Panas?",
  "context": "Petro Yukhymovych Vesklyarov (Ukrainian: Вескляров Петро Юхимович) (June 10 [O.S. May 28] 1911 in Talne, Ukraine – January 5, 1994 in Kyiv) was a Ukrainian theater and television actor. He was also known by the nickname Did Panas (Grandpa Panas, Ukrainian: дід Панас).\nBetween 1932 and 1940, Vesklyarov was an actor in a travelling workers' theatre, and between 1946 and 1959 he performed at the Taras Shevchenko Musical-Drama Theatre in Lutsk, Volyn. Between 1959 and 1982 Veslklyarov worked in the Dovzhenko Film Studios, appearing in a number of films. He starred in the 1959 drama film Ivanna and appeared in the 1970 comedy film Two Days of Miracles. During this time (1964-1986) he appeared as the character \"Дід Панас\" 

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain > 5:llm:AzureChatOpenAI] [1.32s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Grandpa Panas is a nickname for Petro Yukhymovych Vesklyarov, a Ukrainian theater and television actor who appeared as the character \"Дід Панас\" (Grandpa Panas) in the Ukrainian television series \"На добраніч, діти\" (Goodnight, children).",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Grandpa Panas is a nickname for Petro Yukhymovych Vesklyarov, a Ukrainian theater and television actor who appeared as the character \"Дід Панас\" (Grandpa Panas) in the Ukrainian television series \"На добраніч, д

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 6:chain:LLMChain > 7:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "NO_OUTPUT.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "NO_OUTPUT.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 827,
      "completion_tokens": 3,
      "total_tokens": 830
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprint": ""
  },
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 6:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "NO_OUTPUT."
}
[chain/st

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[chain/start] [1:chain:RetrievalQA > 2:retriever:Retriever > 10:chain:LLMChain] Entering Chain run with input:
{
  "question": "Who is Grandpa Panas?",
  "context": "Петро́ Юхи́мович Вескляро́в, ім'я при народженні Пінхас Хаїмович Весклер (9 червня 1911(19110609), Тальне, Уманський повіт, Київська губернія, Російська імперія — 5 січня 1994, Київ) — український актор і телеведучий. Заслужений артист Української РСР (1973). Більш відомий під творчим псевдонімом «Дід Панас».\n\n\n== Життєпис ==\nНародився 9 червня 1911 року в райцентрі Тальне, що на Черкащині. \nПрізвище Петра Юхимовича зазнало змін під час війни з нацизмом: від єврейського Векслер до Вескляров. Це сталося під час перебування в нацистському фільтраційному таборі, щоб приховати єврейське походження.\nПрацював на київській залізниці, де організував театральний гурток, згодом його відправляють до Луцька, у Волинський обласний музично-драматичний театр ім. Тараса Шевченка, де він працював до 1959 року. Там зіграв чимало ролей

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 12:chain:LLMChain > 13:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "NO_OUTPUT.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "NO_OUTPUT.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 571,
      "completion_tokens": 3,
      "total_tokens": 574
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprint": ""
  },
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 12:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "NO_OUTPUT."
}
[chain

In [36]:
with debug_langchain():
    question = "Хто такий дід Панас?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Хто такий дід Панас?"
}
[chain/start] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Хто такий дід Панас?",
  "context": ".\n1941 — призваний у військовий театр Південно-Західного фронту, потрапив у полон, але втік. Згодом потрапив до окупованого гітлерівцями Києва, влаштувався на роботу на залізниці, де організував драматичний гурток. 1946 року був скерований до Луцька — у Волинський український музично-драматичний театр імені Тараса Шевченка, де працював до 1959-го.\nУ Луцьку одружився із секретаркою театру Галиною, де народився його син Богдан.1959—1982 — на Київській кіностудії художніх фільмів імені Олександра Довженка.\nЗ 1962 р. у ролі діда Панаса розповідав на Українському радіо «вечірню казку» для дітей (перед тим Вескляров працював у цій ролі на підміні іншого актора, якого згодом замінив цілком).1964—1986 — у ролі діда Панаса вів 

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain > 5:llm:AzureChatOpenAI] [2.11s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "З 1962 р. у ролі діда Панаса розповідав на Українському радіо «вечірню казку» для дітей (перед тим Вескляров працював у цій ролі на підміні іншого актора, якого згодом замінив цілком). 1964—1986 — у ролі діда Панаса вів на Українському телебаченні програму «На добраніч, діти».",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "З 1962 р. у ролі діда Панаса розповідав на Українському радіо «вечірню казку» для дітей (перед тим Вескляров працював у цій ролі на підміні іншого актора, якого згодом замінив ціл

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 6:chain:LLMChain > 7:llm:AzureChatOpenAI] [2.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Петро́ Юхи́мович Вескляро́в, ім'я при народженні Пінхас Хаїмович Весклер (9 червня 1911(19110609), Тальне, Уманський повіт, Київська губернія, Російська імперія — 5 січня 1994, Київ) — український актор і телеведучий. Заслужений артист Української РСР (1973). Більш відомий під творчим псевдонімом «Дід Панас».",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Петро́ Юхи́мович Вескляро́в, ім'я при народженні Пінхас Хаїмович Весклер (9 червня 1911(19110609), Тальне, Уманський повіт, Київська губернія, Рос

...

[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 8:chain:LLMChain > 9:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "NO_OUTPUT. The context does not provide information about who Дід Панас is.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "NO_OUTPUT. The context does not provide information about who Дід Панас is.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 805,
      "completion_tokens": 19,
      "total_tokens": 824
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprint": ""
  },
  "run": null
}
[chain/end] [1:c

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 10:chain:LLMChain > 11:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Petro Vesklyarov was also known by the nickname Did Panas (Grandpa Panas, Ukrainian: дід Панас).",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Petro Vesklyarov was also known by the nickname Did Panas (Grandpa Panas, Ukrainian: дід Панас).",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 474,
      "completion_tokens": 30,
      "total_tokens": 504
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprin

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 14:chain:StuffDocumentsChain > 15:chain:LLMChain > 16:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Petro Vesklyarov, a Ukrainian actor and TV host who was known by the nickname Did Panas.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Petro Vesklyarov, a Ukrainian actor and TV host who was known by the nickname Did Panas.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 505,
      "completion_tokens": 23,
      "total_tokens": 528
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprint": ""
  

In [37]:
with debug_langchain():
    question = "Що таке розпізнавання іменованих сутностей?"
    # qa_chain({"query": question})
    qa_chain.run(question)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Що таке розпізнавання іменованих сутностей?"
}
[chain/start] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Що таке розпізнавання іменованих сутностей?",
  "context": "У обробці природної мови зв'язування сутностей (англ. Entity Linking), яке також називають зв'язування іменованих сутностей (ЗІС), розпізнавання іменованих сутностей (РІС), розпізнавання і неоднозначності іменованих сутностей (РНІС) або нормалізація іменованих сутностей (НІС), — це присвоєння унікальної ідентичності об'єктам (наприклад, відомим особам, місцям чи компаніям), що згадуються у тексті. Наприклад, розглянемо речення «Дніпро — річка України». Ідея полягає в тому, щоб визначити, що «Дніпро» належить до річки Дніпро, а не до Дніпро-арена чи будь-якої іншої організації, яку можна назвати «Дніпро». Зв'язування сутностей відрізняється від розпізнавання іменованих сутносте

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 4:chain:LLMChain > 5:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Розпізнавання іменованих сутностей (РІС) - це ідентифікація появи іменованої сутності в тексті.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Розпізнавання іменованих сутностей (РІС) - це ідентифікація появи іменованої сутності в тексті.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 764,
      "completion_tokens": 57,
      "total_tokens": 821
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprint": 

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 6:chain:LLMChain > 7:llm:AzureChatOpenAI] [3.80s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Перша фаза, як правило, зводиться до проблеми сегментації: імена визначаються як суміжні проміжки токенів, без вкладеності, таким чином «Банк Америки» є єдиним ім'ям, попри те, що всередині цього імені підрядок «Америки» є іншим ім'ям. Часові вирази та деякі числові вирази (наприклад, гроші, відсотки тощо) також можуть розглядатися як іменовані сутності в контексті завдання РІС.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Перша фаза, як правило, зводиться до проблеми сегментації: імена визначають

...

[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 8:chain:LLMChain > 9:llm:AzureChatOpenAI] [4.21s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Розпізнавання іменованих сутностей (РІС) (також відоме як ідентифікація об'єктної сутності, фрагментація об'єктної сутності та видобуток об'єктної сутності) — це підзадача видобування інформації, яка намагається знайти і класифікувати іменовані сутності в неструктурованому тексті в заздалегідь визначені категорії, такі як імена людей, організації, місця, медичні коди, час, кількості, грошові значення, відсотки тощо.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Розпізнавання іменованих сутностей (Р

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 10:chain:LLMChain > 11:llm:AzureChatOpenAI] [2.41s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "OpenNLP містить в собі засноване на правилах і статистичне розпізнавання іменованих об'єктів. SpaCy має швидке статистичне РІС, а також візуалізатор іменованих сутностей з відкритим вихідним кодом.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "OpenNLP містить в собі засноване на правилах і статистичне розпізнавання іменованих об'єктів. SpaCy має швидке статистичне РІС, а також візуалізатор іменованих сутностей з відкритим вихідним кодом.",
            "additional_kwargs": {}
          }
        }

/opt/homebrew/Caskroom/miniforge/base/envs/llm-simple-QnA-example-1/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 12:chain:LLMChain > 13:llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Що таке розпізнавання іменованих сутностей?",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Що таке розпізнавання іменованих сутностей?",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 383,
      "completion_tokens": 25,
      "total_tokens": 408
    },
    "model_name": "gpt-35-turbo",
    "system_fingerprint": ""
  },
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 2:retriever:Retriever > 12:chain:LLMChain] 